## Get Roads Data and Coordinates

In [ ]:
# get roads in xinyi district 

import pandas as pd

xls = pd.ExcelFile("./tw_raods.xls")

sheetX = xls.parse(0) #2 is the sheet number+1 thus if the file has only 1 sheet write 0 in paranthesis

town_names = sheetX['TownName']
county_id = sheetX['CountyID']
road_names = sheetX['RoadName']

xinyi_roads = []

id = 0
for town in town_names:
    if town == "信義區" and county_id[id] == "A":
        xinyi_roads.append(road_names[id])
        print(road_names[id])
    id += 1

In [ ]:
# load functions for encoding and decoding

from pyproj import Proj, Transformer

# Define the projections for TWD97 TM2 121 and WGS84 using the new syntax
proj_twd97 = Proj('epsg:3826')  # TWD97 TM2 121 projection
proj_wgs84 = Proj('epsg:4326')  # WGS84 projection

# Create a Transformer object for the conversion
transformer = Transformer.from_proj(proj_twd97, proj_wgs84)

def twd97_to_wgs84(x, y):
    # Transform the coordinates from TWD97 to WGS84 using the Transformer object
    lon, lat = transformer.transform(x, y)
    return lat, lon

def decode(encoded_str):
    # Assuming the encoded_str is exactly 8 characters as per your encoding scheme
    # Define the reversed dictionary for decoding
    decoding_dict = {
        '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7,
        '8': 8, '9': 9, 'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14,
        'F': 15, 'G': 16, 'H': 17, 'J': 18, 'K': 19, 'L': 20, 'M': 21,
        'N': 22, 'P': 23, 'Q': 24, 'R': 25, 'S': 26, 'T': 27, 'U': 28,
        'V': 29, 'W': 30, 'X': 31
    }  # Fill in with the reversed mappings

    # Split the encoded string into X and Y components
    encoded_x = encoded_str[:4]
    encoded_y = encoded_str[4:]

    # Pad the Y component to 7 characters
    # encoded_y_padded = encoded_y.ljust(7, '0')

    # Decode each component
    x = 0
    for i, char in enumerate(reversed(encoded_x)):
        x += decoding_dict[char] * (32 ** i)

    y = 0
    for i, char in enumerate(reversed(encoded_y)):
        y += decoding_dict[char] * (32 ** i)
    
    # Add 2,000,000 to the Y component
    y += 2000000
    lon, lat = transformer.transform(x, y)
    return lon, lat

# Example usage of the function with decoded coordinates
x, y = decode("95ELPFWG")  # Use the decode function you provided earlier

print(x, y)

In [ ]:
# get start and end pos on road found
import csv
from xml.etree import ElementTree as ET

file_path = './Section.xml'
tree = ET.parse(file_path)
root = tree.getroot()

namespace = {'ns': 'http://traffic.transportdata.tw/standard/traffic/schema/'}

sections = root.find('ns:Sections', namespace)

section_pos = []

loaded_roads = []

if sections is not None:
    for section in sections:
        road_section = section.find('.//ns:RoadSection', namespace)

        if road_section is not None:
            section_start = road_section.find('.//ns:Start', namespace).text
            section_end = road_section.find('.//ns:End', namespace).text

        road_information = {
            'SectionID': section.find('ns:SectionID', namespace).text,
            'SectionName': section.find('ns:SectionName', namespace).text,
            'RoadID': section.find('ns:RoadID', namespace).text,
            'RoadName': section.find('ns:RoadName', namespace).text,
            'RoadClass': section.find('ns:RoadClass', namespace).text,
            'RoadDirection': section.find('ns:RoadDirection', namespace).text,
            'SectionLength': section.find('ns:SectionLength', namespace).text,
            'RoadSectionStart': section_start,
            'RoadSectionEnd': section_end
        }
        if(road_information['RoadName'] in xinyi_roads):
            loaded_roads.append(road_information['RoadName'])
            section_pos.append([decode(section_start), decode(section_end)])
            print(road_information['RoadName'], decode(section_start), decode(section_end))

with open('./roads_all.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['RoadName', 'Start', 'End'])
    for i in range(len(loaded_roads)):
        writer.writerow([loaded_roads[i], section_pos[i][0], section_pos[i][1]])

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_MAPS_API_KEY = os.environ.get('GOOGLE_MAPS_API_KEY')

In [ ]:
import requests
import math
from geopy.distance import geodesic
from streetview import search_panoramas
from streetview import get_streetview
import googlemaps
import polyline
from shapely.geometry import LineString
import json
import re

def get_address_from_coordinates(latitude, longitude):
    # Google Maps Geocoding API endpoint
    endpoint = "https://maps.googleapis.com/maps/api/geocode/json"
    
    # Your API key (you need to get one from the Google Cloud Console and enable the Geocoding API)
    api_key = GOOGLE_MAPS_API_KEY

    # Parameters for the API request
    params = {
        "latlng": f"{latitude}, {longitude}",
        "key": api_key
    }

    # Send the request and get the response
    response = requests.get(endpoint, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the result
        results = response.json()['results']
        if results:
            # Return the first result (the most relevant address)
            return results[0]['formatted_address']
        else:
            return "No address found for these coordinates."
    else:
        return "Failed to retrieve data."
    
def interpolate_points(start, end, interval_meters):
    distance = geodesic(start, end).meters
    num_intervals = int(math.ceil(distance / interval_meters))
    interpolated_points = [start]
    for i in range(1, num_intervals):
        fraction = i / num_intervals
        interpolated_point = geodesic(kilometers=fraction * (distance / 1000)).destination(start, calculate_heading(*start, *end))
        interpolated_points.append((interpolated_point.latitude, interpolated_point.longitude))
    interpolated_points.append(end)
    return interpolated_points

def calculate_heading(lat1, lon1, lat2, lon2):
    # convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = [math.radians(x) for x in [lat1, lon1, lat2, lon2]]

    # calculate the difference between the longitudes
    dlon = lon2 - lon1

    # calculate the heading using the Haversine formula
    y = math.sin(dlon) * math.cos(lat2)
    x = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(
        dlon
    )
    heading = math.degrees(math.atan2(y, x))

    # normalize the heading to a value between 0 and 360 degrees
    if heading < 0:
        heading += 360

    return heading

In [ ]:
desired_distance = 30  # Example distance

coordinates_list = []

for pos in section_pos:
    origin = get_address_from_coordinates(pos[0][0], pos[0][1])
    destination = get_address_from_coordinates(pos[1][0], pos[1][1])

    gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

    pattern = r'\bXinyi District, Taipei City, Taiwan \d{3}\b'
    directions_result = gmaps.directions(pos[0], pos[1], mode="driving")
    polyline_str = directions_result[0]["overview_polyline"]["points"]
    coords = polyline.decode(polyline_str)
    line = LineString(coords)
    simplified_line = line.simplify(tolerance=0, preserve_topology=False)
    simplified_coords = list(simplified_line.coords)

    final_coords = []

    # Interpolate points along the route at the specified distance intervals
    for i in range(len(simplified_coords) - 1):
        start_coord = simplified_coords[i]
        end_coord = simplified_coords[i + 1]
        final_coords.extend(interpolate_points(start_coord, end_coord, desired_distance))

    for i in range(len(final_coords) - 1):
        pos1 = final_coords[i]
        pos2 = final_coords[i + 1]
        lat, lon = pos1[0], pos1[1]
        heading = calculate_heading(pos1[0], pos1[1], pos2[0], pos2[1])

        addr = get_address_from_coordinates(lat, lon)

        if re.search(pattern, addr):
            print(addr)
            coordinates_list.append({"latitude": lat, "longitude": lon, "heading": heading})
            
json_file_path = os.path.join("./", "coordinates_list.json")

# Write the coordinates list to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(coordinates_list, json_file, indent=4)

print(f"Coordinates have been saved to {json_file_path}")

## Get Street View Photos

In [ ]:
import json

f = open('coordinates_list.json')

data = json.load(f)

# look forward

for i in data:
    lat = i['latitude']
    lon = i['longitude']
    heading = i['heading']
    print(lat, lon)

    panoids = search_panoramas(lat, lon)
    pano = panoids[-1]

    image = get_streetview(
        pano.pano_id, GOOGLE_MAPS_API_KEY, 600, 600, heading,
    )

    filename = f"{lat}_{lon}_fwd.jpg"
    filepath = os.path.join("./images_fwd", filename)
    image.save(filepath, "jpeg")

In [ ]:
import json

f = open('coordinates_list.json')

data = json.load(f)

# look right

for i in data:
    lat = i['latitude']
    lon = i['longitude']
    heading = i['heading']
    print(lat, lon)

    panoids = search_panoramas(lat, lon)
    pano = panoids[-1]

    image = get_streetview(
        pano.pano_id, GOOGLE_MAPS_API_KEY, 600, 600, heading+90,
    )

    filename = f"{lat}_{lon}_r.jpg"
    filepath = os.path.join("./images_right", filename)
    image.save(filepath, "jpeg")

In [ ]:
import json

f = open('coordinates_list.json')

data = json.load(f)

# look left

for i in data:
    lat = i['latitude']
    lon = i['longitude']
    heading = i['heading']
    print(lat, lon)

    panoids = search_panoramas(lat, lon)
    pano = panoids[-1]

    image = get_streetview(
        pano.pano_id, GOOGLE_MAPS_API_KEY, 600, 600, heading-90,
    )

    filename = f"{lat}_{lon}_l.jpg"
    filepath = os.path.join("./images_left", filename)
    image.save(filepath, "jpeg")

In [ ]:
import csv
import pandas as pd

roads = set()
repeated = set()
new_roads = set()

with open('./roads_all.csv', 'r') as f:
    reader = csv.reader(f)
    line_count = 0
    for row in reader:
        if line_count == 0: 
            line_count += 1 
            continue
        roads.add(row[0])
        line_count += 1 

with open('./small_streets.csv', 'r') as f:
    reader = csv.reader(f)
    line_count = 0
    for row in reader:
        if line_count == 0:
            line_count += 1
            continue
        if row[0] not in roads:
            new_roads.add(row[0])
        else:
            repeated.add(row[0])
        line_count += 1

df = pd.read_csv('./small_streets.csv')
for road in repeated:
    df = df.drop(df[df.Name == road].index)
    df.to_csv('./small_streets.csv', index=False)

print(roads)
print(repeated)
print(new_roads)

In [ ]:
import csv
from streetview import search_panoramas
from streetview import get_streetview
import googlemaps
import polyline
from shapely.geometry import LineString
import json
import math
import re
from geopy.distance import geodesic

desired_distance = 30  # Example distance

coordinates_list = []

section_pos = []

with open('./small_streets.csv', 'r') as f:
    reader = csv.reader(f)
    line_count = 0
    for row in reader:
        if line_count == 0:
            line_count += 1
            continue
        start = [row[1].split(',')][0]
        end = [row[2].split(',')][0]
        section_pos.append([start, end])
        line_count += 1

for pos in section_pos:
    origin = get_address_from_coordinates(pos[0][0], pos[0][1])
    destination = get_address_from_coordinates(pos[1][0], pos[1][1])

    gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

    pattern = r'\bXinyi District, Taipei City, Taiwan \d{3}\b'
    directions_result = gmaps.directions(pos[0], pos[1], mode="driving")
    polyline_str = directions_result[0]["overview_polyline"]["points"]
    coords = polyline.decode(polyline_str)
    line = LineString(coords)
    simplified_line = line.simplify(tolerance=0, preserve_topology=False)
    simplified_coords = list(simplified_line.coords)

    final_coords = []

    for i in range(len(simplified_coords) - 1):
        start_coord = simplified_coords[i]
        end_coord = simplified_coords[i + 1]
        final_coords.extend(interpolate_points(start_coord, end_coord, desired_distance))

    for i in range(len(final_coords) - 1):
        pos1 = final_coords[i]
        pos2 = final_coords[i + 1]
        lat, lon = pos1[0], pos1[1]
        heading = calculate_heading(pos1[0], pos1[1], pos2[0], pos2[1])

        addr = get_address_from_coordinates(lat, lon)

        if re.search(pattern, addr):
            print(addr)

            coordinates_list.append({"latitude": lat, "longitude": lon, "heading": heading})

json_file_path = os.path.join("./", "small_streets.json")

# Write the coordinates list to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(coordinates_list, json_file, indent=4)

print(f"Coordinates have been saved to {json_file_path}")

In [ ]:
import json

f = open('small_streets.json')

data = json.load(f)

# look forward

for i in data:
    lat = i['latitude']
    lon = i['longitude']
    heading = i['heading']
    print(lat, lon)

    panoids = search_panoramas(lat, lon)
    pano = panoids[-1]

    image = get_streetview(
        pano.pano_id, GOOGLE_MAPS_API_KEY, 600, 600, heading,
    )

    filename = f"{lat}_{lon}_fwd.jpg"
    filepath = os.path.join("./small_streets/images_fwd", filename)
    image.save(filepath, "jpeg")

In [ ]:
import json

f = open('small_streets.json')

data = json.load(f)

# look right

for i in data:
    lat = i['latitude']
    lon = i['longitude']
    heading = i['heading']
    print(lat, lon)

    panoids = search_panoramas(lat, lon)
    pano = panoids[-1]

    image = get_streetview(
        pano.pano_id, GOOGLE_MAPS_API_KEY, 600, 600, heading+90,
    )

    filename = f"{lat}_{lon}_r.jpg"
    filepath = os.path.join("./small_streets/images_right", filename)
    image.save(filepath, "jpeg")

In [ ]:
import json

f = open('small_streets.json')

data = json.load(f)

# look left

for i in data:
    lat = i['latitude']
    lon = i['longitude']
    heading = i['heading']
    print(lat, lon)

    panoids = search_panoramas(lat, lon)
    pano = panoids[-1]

    image = get_streetview(
        pano.pano_id, GOOGLE_MAPS_API_KEY, 600, 600, heading-90,
    )

    filename = f"{lat}_{lon}_l.jpg"
    filepath = os.path.join("./small_streets/images_left", filename)
    image.save(filepath, "jpeg")

In [ ]:
desired_distance = 30  # Example distance

coordinates_list = []

section_pos = []

with open('./super_small_streets.csv', 'r') as f:
    reader = csv.reader(f)
    line_count = 0
    for row in reader:
        if line_count == 0:
            line_count += 1
            continue
        start = [row[1].split(',')][0]
        end = [row[2].split(',')][0]
        section_pos.append([start, end])
        line_count += 1

for pos in section_pos:
    origin = get_address_from_coordinates(pos[0][0], pos[0][1])
    destination = get_address_from_coordinates(pos[1][0], pos[1][1])

    gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

    pattern = r'\bXinyi District, Taipei City, Taiwan \d{3}\b'
    directions_result = gmaps.directions(pos[0], pos[1], mode="driving")
    polyline_str = directions_result[0]["overview_polyline"]["points"]
    coords = polyline.decode(polyline_str)
    line = LineString(coords)
    simplified_line = line.simplify(tolerance=0, preserve_topology=False)
    simplified_coords = list(simplified_line.coords)

    final_coords = []

    for i in range(len(simplified_coords) - 1):
        start_coord = simplified_coords[i]
        end_coord = simplified_coords[i + 1]
        final_coords.extend(interpolate_points(start_coord, end_coord, desired_distance))

    for i in range(len(final_coords) - 1):
        pos1 = final_coords[i]
        pos2 = final_coords[i + 1]
        lat, lon = pos1[0], pos1[1]
        heading = calculate_heading(pos1[0], pos1[1], pos2[0], pos2[1])

        addr = get_address_from_coordinates(lat, lon)

        if re.search(pattern, addr):
            print(addr)

            coordinates_list.append({"latitude": lat, "longitude": lon, "heading": heading})

json_file_path = os.path.join("./", "super_small_streets.json")

# Write the coordinates list to a JSON file
with open(json_file_path, 'w') as json_file:
    json.dump(coordinates_list, json_file, indent=4)

print(f"Coordinates have been saved to {json_file_path}")

In [ ]:
import folium
import json

# Load your JSON data
with open('./big_roads.json', 'r') as f:
    data1 = json.load(f)

with open('./small_streets.json', 'r') as f:
    data2 = json.load(f)

with open('./super_small_streets.json', 'r') as f:
    data3 = json.load(f)

# Create a base map
# Note: Change latitude and longitude to your desired starting point and adjust zoom_start as needed
map = folium.Map(location=[25.032444, 121.572403], zoom_start=14.25)

# Function to add coordinates from a list to the map
def plot_coordinates(data, map_object, marker_color='blue'):
    for coord in data:
        folium.Marker(
            location=[coord['latitude'], coord['longitude']],
            icon=folium.Icon(color=marker_color),
        ).add_to(map_object)

# Plot coordinates from both JSON files
plot_coordinates(data1, map, 'blue')
plot_coordinates(data2, map, 'red')
plot_coordinates(data3, map, 'green')

# Save or display the map
map.save('map_v2.html')
# Or use map to display inline if using Jupyter Notebook
map

## Convert to JPEG

In [ ]:
import json
from streetview import search_panoramas, get_streetview

f = open('super_small_streets.json')

data = json.load(f)

for i in data:
    lat = i['latitude']
    lon = i['longitude']
    heading = i['heading']
    print(lat, lon)

    panoids = search_panoramas(lat, lon)
    pano = panoids[-1]

    image = get_streetview(
        pano.pano_id, GOOGLE_MAPS_API_KEY, 600, 600, heading,
    )

    filename = f"{lat}_{lon}_fwd.jpg"
    filepath = os.path.join("./super_small_street/images_fwd", filename)
    image.save(filepath, "jpeg")

    image = get_streetview(
        pano.pano_id, GOOGLE_MAPS_API_KEY, 600, 600, heading+90,
    )

    filename = f"{lat}_{lon}_r.jpg"
    filepath = os.path.join("./super_small_street/images_r", filename)
    image.save(filepath, "jpeg")

    image = get_streetview(
        pano.pano_id, GOOGLE_MAPS_API_KEY, 600, 600, heading-90,
    )

    filename = f"{lat}_{lon}_l.jpg"
    filepath = os.path.join("./super_small_street/images_l", filename)
    image.save(filepath, "jpeg")

## Record Coordinates to CSV File

In [ ]:
import os
import csv

def extract_coordinates(filename):
    base_name = filename[:-3:]
    coordinate = '_'.join(base_name.split('_')[:2])
    return coordinate

def record_coordinates(folder_path, output_file='coordinates.csv'):
    try:
        # Set to store unique coordinates
        unique_coordinates = set()

        # Walk through the directory
        for dirpath, dirnames, filenames in os.walk(folder_path):
            for filename in filenames:
                coordinate = extract_coordinates(filename)
                print(coordinate)
                unique_coordinates.add(coordinate)

        # Write the unique coordinates to the CSV file
        with open(output_file, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['lat', 'lon'])

            for coordinate in sorted(unique_coordinates):
                lat, lon = coordinate.split('_')
                csvwriter.writerow([lat, lon])
        
        print(f"Coordinates have been recorded in {output_file}.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Replace 'your_folder_path' with the path to your folder
folder_path = './photos'
record_coordinates(folder_path)


## Record Photos URL to CSV File

In [ ]:
import pandas as pd

def get_image_url(lat, lon):
    fwd = "https://github.com/hhe1ibeb/xinyi_geosearch/blob/dev/data/photos/" + str(lat) + "_" + str(lon) + "_fwd.jpeg?raw=true"
    l = "https://github.com/hhe1ibeb/xinyi_geosearch/blob/dev/data/photos/" + str(lat) + "_" + str(lon) + "_l.jpeg?raw=true"
    r = "https://github.com/hhe1ibeb/xinyi_geosearch/blob/dev/data/photos/" + str(lat) + "_" + str(lon) + "_r.jpeg?raw=true" 
    return [fwd, l, r]

df = pd.read_csv('coordinates.csv')

for index in range(len(df)):
    lat = df['lat'][index]
    lon = df['lon'][index]
    urls = get_image_url(lat, lon)
    df.at[index, 'fwd'] = urls[0]
    df.at[index, 'l'] = urls[1]
    df.at[index, 'r'] = urls[2]

df.to_csv('coordinates.csv', index=False)

In [2]:
import pandas as pd

# Load the CSV file
file_path = '../coordinates_translated.csv'
df = pd.read_csv(file_path)

# Combine 'lat' and 'lon' columns into a single 'coordinates' column
df['coordinates'] = df['lat'].astype(str) + ', ' + df['lon'].astype(str)

# Save the modified DataFrame to a new CSV file
safe_file_path = '../coordinates_safe.csv'
df.to_csv(safe_file_path, index=False)

print(f"Modified file saved to: {safe_file_path}")

Modified file saved to: ../coordinates_safe.csv
